In [29]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
%matplotlib notebook
import matplotlib.pyplot as plt

def plt_dynamic(epoch_list, avg_train_loss_list, avg_test_loss_list, ax,ticks,title):
    ax.plot(epoch_list, avg_train_loss_list, 'b', label="Train Loss")
    ax.plot(epoch_list, avg_test_loss_list, 'r', label="Test Loss")
    if len(epoch_list)==1:
        plt.legend()
        plt.title(title)
    plt.yticks(ticks)
    fig.canvas.draw()

In [31]:
# Network Parameters
n_hidden_1 = 512 # 1st layer number of neurons
n_hidden_2 = 128 # 2nd layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [32]:
# placeholders for mini-batch dataset
x_is = tf.placeholder(tf.float32, [None, 784])
y_true_is = tf.placeholder(tf.float32, [None, 10])

In [33]:
# this would be used for dropouts in dropouts section

# keep_prob: we will be using these placeholders when we use dropouts, while testing model
keep_prob = tf.placeholder(tf.float32)
# keep_prob_input: we will be using these placeholders when we use dropouts, while training model
keep_prob_input = tf.placeholder(tf.float32)

In [34]:
# Weight initialization

# sgd means sigmoidal unit. So this is the initialization for sigmoidal activation units
# SGD: Xavier/Glorot Normal initialization.
weights_sgd = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=0.039, mean=0)),    #784x512 
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=0.055, mean=0)), #512x128
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=0.120, mean=0))  #128x10
}

# for relu activation unit
# He Normal initialization.
weights_relu = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=0.0505, mean=0)),    #784x512
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=0.0624, mean=0)), #512x128
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=0.125, mean=0))  #128x10
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),             #512x1
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),             #128x1
    'out': tf.Variable(tf.random_normal([n_classes]))              #10x1
}

In [35]:
# Parameters
training_epochs = 15
learning_rate = 0.001  # this is value of eeettaa i.e if we use SGD optimiser then this would be fixed.
batch_size = 100 # mini-batch size for each iteration.
display_step = 1 # it is to tell, after how many epochs interval, you want to update the graph of loss Vs epochs.

<h3>STEPS :<br>
<p>    
   a) Define your y_hat_is in terms of W's , x's and in-between layers by defining your multilayer_perceptron.<br><br>
   b) Define your loss function in terms of y_hat_is and y_is.<br><br>
   c) Define/Choose the optimiser which would reduce the loss which you defined by improving W's of each layer.<br><br>
   d) In order to start training i.e in order to update W's for each layer, you have to pass your optimiser inside the sess.run() method by filling the memory of placeholders i.e by sending mini-batch dataset for x_is and y_true_is.
</p>
</h3>

<h3> Model 1: input (784) - sigmoid(512) - sigmoid(128) - 
softmax(output 10) </h3>

In [43]:
# Create model (this is the place where you define architecture of layers and activation units in each of those layers)
def multilayer_perceptron(x, weights, biases):
    # Use tf.matmul instead of "*" because tf.matmul can change it's dimensions on the fly (broadcast)
    print( 'x:', x.get_shape(), 'W[h1]:', weights['h1'].get_shape(), 'b[h1]:', biases['b1'].get_shape())        
    
    # Hidden layer with Sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) #(x*weights['h1']) + biases['b1']
    layer_1 = tf.nn.sigmoid(layer_1)
    print( 'layer_1:', layer_1.get_shape(), 'W[h2]:', weights['h2'].get_shape(), 'b[h2]:', biases['b2'].get_shape())        
    
    # Hidden layer with Sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) # (layer_1 * weights['h2']) + biases['b2'] 
    layer_2 = tf.nn.sigmoid(layer_2)
    print( 'layer_2:', layer_2.get_shape(), 'W[out]:', weights['out'].get_shape(), 'b3:', biases['out'].get_shape())        
    
    # Output layer with Softmax activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] # (layer_2 * weights['out']) + biases['out']    
    out_layer = tf.nn.softmax(out_layer)
    print('out_layer:',out_layer.get_shape())

    return out_layer  

<h5> __ Model 1 + AdamOptimizer __ (Sigmoid + Adam) </h5>

In [45]:
# Since we are using sigmoid activations in hiden layers we will be using weights that are initalized as weights_sgd
y_hat_sgd = multilayer_perceptron(x_is, weights_sgd, biases) # this is final y_hat_is.

# this is loss function for sigmoid activation units, in hidden layers.
cost_sgd = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_hat_sgd, labels = y_true_is))

# there are many other optimizers available (these are the optimisers, which would reduce cost_sgd)
optimizer_adam = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost_sgd)
optimizer_sgdc = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost_sgd)

# ? means that you have to pass mini-batch dataset, at the time of actual training.

x: (?, 784) W[h1]: (784, 512) b[h1]: (512,)
layer_1: (?, 512) W[h2]: (512, 128) b[h2]: (128,)
layer_2: (?, 128) W[out]: (128, 10) b3: (10,)
out_layer: (?, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
with tf.Session() as sess:
    
    # initializing all the weights and biases i.e of all the layers
    tf.global_variables_initializer().run()
    
     # for plotting graph
    fig,ax = plt.subplots(1,1)
    ax.set_xlabel('epoch') ; ax.set_ylabel('Soft Max Cross Entropy loss')
    epochs_list, avg_train_loss_list, avg_test_loss_list = [], [], []
    
    for epoch in range(training_epochs):
        train_avg_cost = 0.
        test_avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)

        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)

            # here c: corresponds to the parameter cost_sgd
            # w : corresponds to the parameter weights_sgd
            # c = sess.run() returns the cost after every batch during train --> this is training error after the iteration
            # w = sess.run() returns the weights that are modified after every batch through Back prop
            # w is dict w = {'h1': updated h1 weight vector after the current batch,
            #                'h2': updated h2 weight vector after the current batch, 
            #                'out': updated output weight vector after the current batch, 
            #                }
            # you check these w matrix for every iteration, and check whats happening during back prop
            #
            # note: sess.run() returns parameter values based on the input parameters
            # feed_dict={x_is: batch_xs, y_true_is: batch_ys} here x_is, y_true_is should be placeholders
            # x_is, y_true_is are the input parameters on which the model gets trained.

            # here we use AdamOptimizer
            _, c, w = sess.run([optimizer_adam, cost_sgd,weights_sgd], feed_dict={x_is: batch_xs, y_true_is: batch_ys})
            train_avg_cost += c / total_batch
            c = sess.run(cost_sgd, feed_dict={x_is: mnist.test.images, y_true_is: mnist.test.labels})
            test_avg_cost += c / total_batch

        epochs_list.append(epoch)
        avg_train_loss_list.append(train_avg_cost)
        avg_test_loss_list.append(test_avg_cost)
        plt_dynamic(epochs_list, avg_train_loss_list, avg_test_loss_list, ax, np.arange(1.3, 1.8, step=0.04), "input-sigmoid(512)-sigmoid(128)-sigmoid(output)-AdamOptimizer")

        if epoch%display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "train cost={:.9f}".format(train_avg_cost), "test cost={:.9f}".format(test_avg_cost))
    plt_dynamic(xs, ytrs, ytes, ax, np.arange(1.3, 1.8, step=0.04), "input-sigmoid(512)-sigmoid(128)-sigmoid(output)-AdamOptimizer")

    # we are calculating the final accuracy on the test data
    correct_prediction = tf.equal(tf.argmax(y_sgd,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))